# **Aprendizaje Automático - Clasificación**

- Francisco Prados Abad
- Paola León Tarife
- Julia de Enciso García
- Paula Samper López
- Camino Rodríguez Pérez-Carral
- Lucía Yan Wu


****

In [ ]:
# Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_absolute_error, r2_score,accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns

# CLasificacion
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

from random import randint

# Set seed
np.random.seed(42)

import joblib
from sklearn.neural_network import MLPClassifier

### Cargar los datos

In [ ]:
# Accedemos a las bases de datos procesadas
X_train = pd.read_csv('data/clasificación/X_train.csv')
y_train = pd.read_csv('data/clasificación/y_train.csv')
X_test = pd.read_csv('data/clasificación/X_test.csv')
y_test = pd.read_csv('data/clasificación/y_test.csv')

## **Modelos de clasificación**

Tras el preprocesado de los datos, se probarán distintos modelos de aprendizaje automático centrados en la tarea de clasificación.
Previamente a la implementación de cada uno de los modelos, se analizó el objetivo y a qué tipo de conjunto de datos suele aplicarse, centrándonos en:
-	**Interpretabilidad** en nuestro caso, pero, generalmente en este tipo de casos de concesión de un crédito, los modelos interpretables son más valiosos con el objetivo de justificar por qué se ha aprobado o no dicho crédito (LR/DT)
-**Comportamiento lineal/no lineal**: lineal (LR, LDA) no lineal (DT, KNN, SVC, NN). Entendemos que nuestro caso las relaciones son más complejas y no necesariamente lineales.
-	**Ruido y datos desbalanceados:** por ejemplo el Naive Bayes, al asumir independencia entre características, puede que no sea la mejor opción. El problema de datos desbalanceados se solventó durante el preprocesamiento.
-	**Cantidad de datos y dimensionalidad:** para gran volumen de datos (SGD, NN), y el KNN puede ser ineficientes con muchos datos.
-	**Velocidad de predicción**


### Árboles de Decisión

El **DecisionTreeClassifier** construye un árbol de decisión para clasificar datos dividiéndolos recursivamente en subconjuntos en función de las características de entrada. Cada nodo del árbol representa una decisión basada en una característica específica, y las hojas finales representan las clases de salida.



*   **max_depth:** Profundidad máxima del árbol. Controla cuántos niveles tendrá el árbol. Profundidades más grandes tienden a sobreajustar el modelo. Valores a probar: 3, 5, 7.
*  **max_features:** Proporción de características a considerar en cada división. Valores a probar: 0.25, 0.5, 0.75, 1.0.
*   **min_samples_leaf:** Número mínimo de muestras que debe contener una hoja. Hojas con menos muestras de las indicadas no se crearán, lo que puede evitar el sobreajuste.
Valores a probar: 2, 3, 4, 5, 6.

*   **criterion:** Función para medir la calidad de una división.
'gini': Índice de Gini. 'entropy': Basado en la ganancia de información (entropía).








In [ ]:
# Crear un clasificador de árbol de decisión
arbol_decision = DecisionTreeClassifier(random_state=42, min_samples_split=12)

# Definir distintos valores de los parámetros
param_grid_tree = {"max_depth": [3,5,7],
            "max_features": [0.25, 0.5, 0.75, 1.0],
            "min_samples_leaf": [2,3,4,5,6],
            "criterion": ["gini","entropy"],
            }

arbol_decision_random = RandomizedSearchCV(estimator = arbol_decision, param_distributions = param_grid_tree, n_iter = 20, scoring = 'roc_auc', cv = 5, verbose=False)
arbol_decision_random.fit(X_train, y_train.values.ravel())

# Entrenar el modelo con los datos de entrenamiento
arbol_decision_random.fit(X_train, y_train)

# Hacer predicciones con los datos de prueba
y_pred_tree = arbol_decision_random.predict(X_test)

# Evaluar la precisión del modelo
precision_tree = roc_auc_score(y_test, y_pred_tree)
print("Mejor combinación de hiperparámetros:", arbol_decision_random.best_params_)
print("AUC:", precision_tree)

Mejor combinación de hiperparámetros: {'min_samples_leaf': 2, 'max_features': 0.75, 'max_depth': 7, 'criterion': 'entropy'}
AUC: 0.868889592099444


### Regresión logística

La **Regresión Logística** es un modelo lineal utilizado para problemas de clasificación binaria. Estima la probabilidad de que una instancia pertenezca a una clase, utilizando la función sigmoide para transformar la salida lineal en una probabilidad entre 0 y 1.

- **penalty:** penalización regularizadora que controla la magnitud de los coeficientes. l2 = problemas lineales
- **C:** inverso regularización, controla la magnitud de la penalización aplicada. A mayor valor de C, se reduce la regularización.
- **solver:** optimizador. liblinear = datasets pequeños // saga = l1/elasticnet
- **max_iter:** si el dataset es grande, conviene aumentarlo

In [ ]:
l_regresion = LogisticRegression(random_state=42, class_weight = 'balanced')

# Definir distintos valores de los parámetros
param_grid_lr = {
    'penalty': ['l1', 'l2', 'elasticnet'],    # Varias opciones de regularización
    'C': [0.001, 0.01, 0.1, 1, 10, 100],      # Un rango amplio para la regularización
    'solver': ['liblinear', 'saga'],           # Solvers compatibles con l1 y elasticnet
    'max_iter': [100, 200, 300]                # Más iteraciones si el dataset es grande
}

l_regresion_random = RandomizedSearchCV(estimator = l_regresion, param_distributions = param_grid_lr, n_iter = 20, scoring = 'roc_auc', cv = 5, verbose=False)
l_regresion_random.fit(X_train, y_train.values.ravel())

y_pred_lr = l_regresion_random.predict(X_test)

# Calcular la precisión usando el AUC
precision_lr = roc_auc_score(y_test, y_pred_lr)
print("Mejor combinación de hiperparámetros:", l_regresion_random.best_params_)
print("AUC:", precision_lr)

/home/pvltarife/miniconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
35 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/home/pvltarife/miniconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/pvltarife/miniconda3/lib/python3.11/site-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/pvltarife/miniconda3/lib/python3.11/site-packages/sklearn/linear_

Mejor combinación de hiperparámetros: {'solver': 'liblinear', 'penalty': 'l1', 'max_iter': 200, 'C': 0.1}
AUC: 0.9211876200286613


### Gaussian Naive Bayes

**GaussianNB** es una variante del clasificador Naive Bayes que asume que las características siguen una distribución gaussiana (normal). Este modelo es rápido y efectivo para problemas de clasificación donde los datos tienen esta distribución, aunque puede no ser adecuado si las distribuciones de las características son complejas o multimodales. En este caso, no esperaríamos que devolviese buenos resultados ya que nuestros datos no siguen una distribución gaussiana, pero se probó igualmente.

- **var_smoothing:** permite controlar la varianza de la suavización aplicada a evitar que las probabilidades estimadas sean exactamente cero. Para features cercanas a 0.

In [ ]:
gnb = GaussianNB()

# Definir los valores del hiperparámetro a buscar
param_grid_gnb = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]  # Diferentes niveles de suavización
}

# Implementar GridSearchCV con los parámetros definidos
gnb_random = RandomizedSearchCV(estimator=gnb, param_distributions=param_grid_gnb, n_iter = 20, scoring='roc_auc', cv=5, verbose=1)

# Entrenar el modelo con los datos de entrenamiento
gnb_random.fit(X_train, y_train.values.ravel())

# Hacer predicciones en los datos de test
y_pred_gnb = gnb_random.predict(X_test)

# Calcular la precisión usando el AUC
precision_gnb = roc_auc_score(y_test, y_pred_gnb)
print("Mejor combinación de hiperparámetros:", gnb_random.best_params_)
print("AUC:", precision_gnb)


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 5 is smaller than n_iter=20. Running 5 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 5 candidates, totalling 25 fits
Mejor combinación de hiperparámetros: {'var_smoothing': 1e-05}
AUC: 0.8688764252227914


### KNN

El clasificador **K-Nearest Neighbors (KNN)** es un modelo basado en instancias que clasifica un nuevo punto según la clase de los k vecinos más cercanos en el espacio de características. El rendimiento del modelo depende en gran medida de la elección del número de vecinos (k) y el método de distancia. De ahí que fueran uno de los principales hiperparámetros a buscar.


*   **n_neighbors:** Número de vecinos a considerar para la clasificación. Valores a probar: 5, 10, 20, 30, 40, 50.

*  **weights:** Cómo se ponderan los vecinos.'uniform': Todos los vecinos tienen el mismo peso.'distance': Los vecinos más cercanos tienen más peso.

*  **algorithm:** Algoritmo para computar los vecinos más cercanos.
'auto': Elige automáticamente el mejor algoritmo según los datos. 'ball_tree', 'kd_tree', 'brute': Diferentes estructuras de datos para búsquedas rápidas de vecinos.
*   **leaf_size:** Tamaño de la hoja para los árboles ball_tree y kd_tree. Afecta la velocidad de construcción y consulta de árboles.








In [ ]:
knn = KNeighborsClassifier(algorithm = 'brute', n_jobs=-1)

# Definir distintos valores de los parámetros
param_grid_knn = {'n_neighbors': [5, 10, 20, 30, 40, 50],
               'weights': ['uniform', 'distance'],
               'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
               'leaf_size': [10, 20, 30, 40, 50]
               }
knn_random = RandomizedSearchCV(estimator = knn, param_distributions = param_grid_knn, n_iter = 20, scoring = 'roc_auc', cv = 5, verbose=False)
knn_random.fit(X_train, y_train.values.ravel())

y_pred_knn = knn_random.predict(X_test)

precision_knn = roc_auc_score(y_test, y_pred_knn)
print("Mejor combinación de hiperparámetros:", knn_random.best_params_)
print("AUC", precision_knn)

Mejor combinación de hiperparámetros: {'weights': 'distance', 'n_neighbors': 20, 'leaf_size': 10, 'algorithm': 'brute'}
AUC 0.8991934640699143


### SVC

El **LinearSVC** es una implementación lineal de las máquinas de vectores de soporte (SVM). Se utiliza principalmente para problemas de clasificación binaria, buscando un hiperplano que separe las clases de manera óptima, de ahí que lo hayamos escogido en nuestro problema. Utiliza penalización L2 para regularización y admite varias funciones de pérdida.


*   **penalty:** Regularización aplicada. Solo admite 'l2' en LinearSVC.

*   **loss:** Función de pérdida.'hinge': Pérdida de margen duro (SVM clásico).'squared_hinge': Pérdida cuadrada del margen.
*   **C:** Parámetro de regularización (inverso de la fuerza de regularización). Valores más grandes permiten menos regularización.
Valores a probar: 0.0001, 0.001, 0.01, 0.1, 1, 10, 100.
*   **max_iter:** Número máximo de iteraciones para la convergencia del algoritmo.Valores a probar: 1000, 5000, 10000.
*   **tol:** Tolerancia para detener el criterio de convergencia. Valores a probar: 1e-4, 1e-3, 1e-2.
*   **dual:** Resolver el problema dual (True) o primario (False). En datasets con muchas características, dual=False puede ser más eficiente.










In [ ]:
svm = LinearSVC(random_state=42, class_weight='balanced')
param_grid_svm = {
    'penalty': ['l2'],                           # Penalización l2 (la única disponible en LinearSVC)
    'loss': ['hinge', 'squared_hinge'],          # Tipos de pérdida
    'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100], # Regularización
    'max_iter': [1000, 5000, 10000],             # Número máximo de iteraciones
    'tol': [1e-4, 1e-3, 1e-2],                  # Tolerancia para la convergencia
    'dual': [True, False]                        # Resolver el problema en su forma dual o primal (para datasets con muchas features = False)
 }

svm_random = RandomizedSearchCV(estimator = svm, param_distributions = param_grid_svm, n_iter = 20, scoring = 'roc_auc', cv = 5, verbose=False)
svm_random.fit(X_train, y_train.values.ravel())

y_pred_svm = svm_random.predict(X_test)

precision_svm = roc_auc_score(y_test, y_pred_svm)
print("Mejor combinación de hiperparámetros:", svm_random.best_params_)
print("AUC", precision_svm)


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  w

Mejor combinación de hiperparámetros: {'tol': 0.001, 'penalty': 'l2', 'max_iter': 1000, 'loss': 'squared_hinge', 'dual': True, 'C': 0.1}
AUC 0.9202466891290162


### SGD

El **SGDClassifier** es una implementación de clasificación lineal que utiliza descenso por gradiente estocástico. Es eficiente para grandes conjuntos de datos, ya que actualiza los pesos de manera iterativa con mini-batches de datos, en lugar de procesar todo el dataset de una vez. Como nuetra dimensión es relativamente alta, se decidió probar este modelo.



*  **penalty:** Regularización L2 para evitar el sobreajuste.

*   **loss:** Función de pérdida.'hinge': Pérdida de margen duro (como en SVM).'squared_hinge': Pérdida cuadrada de margen (más suave).
*   **alpha:** Parámetro de regularización (fuerza de la penalización).
Valores a probar: 1e-6, 1e-4, 1e-3, 1e-2, 0.1, 1.

*   **max_iter:** Número máximo de iteraciones para convergencia.Valores a probar: 10, 100, 1000, 5000, 10000.

*   **tol:** Tolerancia para la convergencia del optimizador. Valores a probar: 1e-4, 1e-3, 1e-2.









In [ ]:
### SGD
sgd = SGDClassifier(random_state=42)
param_grid_sgd = {
    'penalty': ['l2'],                          # Penalización L2
    'loss': ['hinge', 'squared_hinge'],          # Función de pérdida
    'alpha': [1e-6, 1e-4, 1e-3, 1e-2, 0.1, 1],  # Parámetro de regularización
    'max_iter': [10, 100, 1000, 5000, 10000],   # Número máximo de iteraciones
    'tol': [1e-4, 1e-3, 1e-2]                   # Tolerancia para la convergencia
}

sgd_random = RandomizedSearchCV(estimator = sgd, param_distributions = param_grid_sgd, n_iter = 20, scoring = 'roc_auc', cv = 5, verbose=False)
sgd_random.fit(X_train, y_train.values.ravel())

# Definir el clasificador SGD
# Hacer predicciones en los datos de test
y_pred_sgd = sgd_random.predict(X_test)

# Calcular la precisión usando el AUC
precision_sgd = roc_auc_score(y_test, y_pred_sgd)
print("Mejor combinación de hiperparámetros:", sgd_random.best_params_)
print("AUC:", precision_sgd)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:744: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:744: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:744: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:744: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic

Mejor combinación de hiperparámetros: {'tol': 0.001, 'penalty': 'l2', 'max_iter': 10000, 'loss': 'hinge', 'alpha': 0.001}
AUC: 0.9217728009752576


### LDA

El **Linear Discriminant Analysis (LDA)** es un clasificador lineal que busca proyectar los datos en un espacio de menor dimensión, maximizando la separación entre las clases. Es útil en problemas donde las clases son linealmente separables. En este caso, no es la caraterística más representable, aun así, se decidió probar este modelo para ver qué resultados ofrecía.



*  **solver:** Método de resolución.'svd': No aplica regularización.'lsqr': Resuelve el problema de forma rápida y puede aplicar regularización.'eigen': Similar a 'lsqr', pero basado en descomposición de valores propios.
*   **priors:** Proporciones a priori de las clases. Si no se especifica, se asume que las clases están balanceadas.
*   **shrinkage:** Regularización aplicada a la estimación de la covarianza. Solo para 'lsqr' y 'eigen'.'auto': Selecciona automáticamente el parámetro de regularización.
*  **store_covariance:** Si se debe almacenar la matriz de covarianza del modelo.







In [ ]:
### LDA
lda = LinearDiscriminantAnalysis()
# Definir los valores de los hiperparámetros a buscar
param_grid_lda = {
    'solver': ['svd', 'lsqr', 'eigen'],                 # Métodos de resolución
    'priors': [None, [0.5, 0.5], [0.3, 0.7]],            # Proporciones a priori de las clases (útil cuando conjuntos desbalanceados)
    'shrinkage': [None, 'auto'],                         # Regularización (solo para 'lsqr' y 'eigen')
    'storeovariance': [True, False]                    # Almacenar la matriz de covarianza
}
lda_random = RandomizedSearchCV(estimator = lda, param_distributions = param_grid_lda, n_iter = 20, scoring = 'roc_auc', cv = 5, verbose=False)
lda_random.fit(X_train, y_train.values.ravel())

# Hacer predicciones en los datos de test
y_pred_lda = lda_random.predict(X_test)

# Calcular la precisión usando el AUC
precision_lda = roc_auc_score(y_test, y_pred_lda)
print("Mejor combinación de hiperparámetros:", lda_random.best_params_)
print("AUC:", precision_lda)

Mejor combinación de hiperparámetros: {'store_covariance': False, 'solver': 'eigen', 'shrinkage': 'auto', 'priors': [0.3, 0.7]}
AUC: 0.8534073482799613


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
20 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/discriminant_analysis.py", line 629, in fit
    raise NotImplementedError("shrinkage not supported with 'svd' solver.")
N

Finalmente, podemos apreciar que el modelo con mejor resultado es el

## **Redes neuronales**

Tras evaluar los modelos de aprendizaje automático previos ajustando los mejores hiperparámetros para cada uno de ellos, procedemos a probar con la implementación de redes neuronales sencillas.

Las **redes neuronales** son modelos computacionales inspirados en el cerebro humano, que consisten en múltiples capas de neuronas artificiales interconectadas. Suelen ser muy efectivas en problemas de clasificación como el nuestro. Las redes neuronales aprenden ajustando los pesos de las conexiones entre neuronas mediante un proceso iterativo de optimización llamado backpropagation.

Además, son capaces de capturar patrones complejos y relaciones no lineales presentes en los datos.

### MLP

Se implementó una **Red Multiceptron (MLP)** que es un tipo de red neuronal feedforward compuesta por al menos tres capas: una capa de entrada, una o más capas ocultas, y una capa de salida.

Cada neurona en una capa está conectada con todas las neuronas de la siguiente capa, lo que permite al MLP capturar **patrones no lineales complejos**.

Para nuestra tarea, se probó a implementar en primer lugar, una MLP sencilla para ver cómo actuaba en comparación con los modelos de aprendizaje automático previos.

Para la definición del espacio de búsqueda de los hiperparámetros, primero definimos qué realiza cada uno de ellos:


*   **hidden_layer_sizes:** especifica la arquitectura, es decir, el número de neuronas en cada capa oculta.
*   **activation:** la función de activación que se aplicará a las neuronas. Entre los posibles valores tenemos 'tanh' y 'relu', esta última con eficiencia en problemas no lineales.

*  **solver:** especifica el algoritmo de optimización para ajustar los pesos. Las posibles opciones son 'sgd' y 'adam', siendo este último generalmente más rápido y eficaz en muchas situaciones.
*  **alpha:** Parámetro de regularización L2 (penalización de los pesos para evitar el sobreajuste).Cuyos valores posibles fueron o 0.0001, o 0.05, entre otros que se probaron.
* **learning_rate:** Define la tasa de aprendizaje que afecta cómo se ajustan los pesos con cada iteración. Pudiendo ser fija 'constant' o adaptativa según la iteración 'adaptative'.

In [ ]:
from sklearn.neural_network import MLPClassifier
### MLP
mlp = MLPClassifier(random_state=42, max_iter=100)
# Definir los valores de los hiperparámetros a buscar
param_grid_mlp = {
    'hidden_layer_sizes': [(10,30,10),(20,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}
mlp_random = RandomizedSearchCV(estimator = mlp, param_distributions = param_grid_mlp, n_iter = 20, scoring = 'roc_auc', cv = 5, verbose=False)
mlp_random.fit(X_train, y_train.values.ravel())

# Hacer predicciones en los datos de test
y_pred_mlp = mlp_random.predict(X_test)

# Calcular la precisión usando el AUC
precision_mlp = roc_auc_score(y_test, y_pred_mlp)
print("Mejor combinación de hiperparámetros:", mlp_random.best_params_)
print("AUC:", precision_mlp)

/home/pvltarife/miniconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/pvltarife/miniconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/pvltarife/miniconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/pvltarife/miniconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/pvltarife/mini

Mejor combinación de hiperparámetros: {'solver': 'adam', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (10, 30, 10), 'alpha': 0.0001, 'activation': 'tanh'}
AUC: 0.9258830161850472


/home/pvltarife/miniconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


Los mejores hiperparámetros obtenidos fueron: **'adam'**, que como antes se adelantaba suele ser más eficaz. Por otro lado, una tasa de aprendizaje **adaptativa**, de esa forma, varía según la iteración ajustándose a nuestro problema.
Por último, resaltar que la arquitectura de la MLP escogida está constituida por una red de **tres capas ocultas** con 10, 30 y 10 neuronas respectivamente, aplicándose en cada una de ellas una función **tangente hiperbólica**.

### Keras

A continuación, visto que la MLP nos proporcionó un valor superior a todo lo anterior visto, se decidió probar e investigar otro tipos de redes mediante la API de **Keras**.

Keras, una API de alto nivel para redes neuronales, permite implementar fácilmente un MLP. Se define el modelo secuencialmente añadiendo capas densas (Dense) para cada capa oculta y de salida. La optimización del modelo se realiza utilizando optimizadores como adam o sgd, y se ajustan hiperparámetros como la tasa de aprendizaje, el número de capas y neuronas, el tipo de función de activación, y otros parámetros. Keras simplifica el proceso de construir, entrenar y evaluar redes neuronales.

Asimismo, se implementó también una búsqueda de hiperparámetros con **Optuna**. Optuna es una herramienta para la optimización automática de hiperparámetros. En el caso de redes neuronales implementadas con Keras, Optuna puede explorar diferentes combinaciones de hiperparámetros, como el número de neuronas en cada capa, el optimizador, la tasa de aprendizaje, y el número de capas ocultas. Optuna utiliza técnicas avanzadas de búsqueda para encontrar las configuraciones óptimas, mejorando el rendimiento del modelo en menos tiempo.


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow
tensorflow.random.set_seed(42)
np.random.seed(42)
keras.utils.set_random_seed(42)

#### Keras simple

In [ ]:

# Define the Neural Network model
model = keras.Sequential([
    layers.Input(shape=(X_train.shape[1],)),  # Input layer
    layers.Dense(64, activation='relu'),  # Hidden layer
    layers.Dense(32, activation='relu'),  # Another hidden layer
    layers.Dense(1)  # Output layer
])
# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)  # Adjust epochs and batch_size as needed

# Make predictions
y_pred = model.predict(X_test)

# Calculate Mean Absolute Error
auc = roc_auc_score(y_test, y_pred)
print(f"AUC for y_pred: {auc}")

Epoch 1/100
1759/1759 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 0.2028
Epoch 2/100
1759/1759 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.0877
Epoch 3/100
1759/1759 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 0.0773
Epoch 4/100
1759/1759 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.0737
Epoch 5/100
1759/1759 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.0715
Epoch 6/100
1759/1759 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.0699
Epoch 7/100
1759/1759 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0692
Epoch 8/100
1759/1759 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 0.0683
Epoch 9/100
1759/1759 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0679
Epoch 10/100
1759/1759 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0674
Epoch 11/100
1759/1759 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 0.0668
Epoch 12/100
1759/1759 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 0.0666
Epoch 13/100
1759/1759 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.0664
Epoch 14/100
1759/1759 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0663
Epoch 15/100
17

Con una implementación más sencilla, vemos que supera a la MLP anteriormente implementada, por lo tanto, se aspira a encontrar una mejor red neuronal al utilizar Optuna para encontrar los mejores hiperparámetros.

### Keras + Optuna

In [ ]:
!pip install --quiet optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.7 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from optuna.samplers import TPESampler
import random

def set_random_seed(seed=42):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    random.seed(seed)

set_random_seed(42)

Para utilizar Optuna necesitamos un conjunto de validación, para ello, se extrae el 20% de los datos del conjunto de entrenamiento.

In [ ]:
# División en conjuntos de train, test y validación
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=y_train, test_size=0.2, random_state= 42)

In [ ]:
import optuna
from tensorflow import keras
from keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report

La siguiente configuración propone a Optuna elegir el mejor número de capas ocultas, así como el número de neuronas de cada una de ellas.
Así mismo, se decide buscar la tasa de aprendizaje de los distintos regularizadores.

In [ ]:

def create_model(trial):

    model = Sequential()

    num_hidden_layers = trial.suggest_int('num_hidden_layers', 1,3)

    input_shape = (X_train.shape[1],)
    model.add(Input(shape=input_shape))

    for i in range(num_hidden_layers):
        units_i = trial.suggest_categorical(f'units_{i+1}', [2**i for i in range(4, 8)])
        model.add(Dense(units_i, activation='relu', kernel_initializer='random_normal', kernel_regularizer=regularizers.L2(trial.suggest_float(f'lr_l2_{i+1}',  1e-5, 1e-2, log=True)), bias_regularizer=regularizers.L2(trial.suggest_float(f'lr_l2_{i+1}',  1e-5, 1e-2, log=True)),activity_regularizer=regularizers.L2(trial.suggest_float(f'lr_l2_{i+1}',  1e-5, 1e-2, log=True))))
        model.add(Dropout(trial.suggest_float(f'dropout_{i+1}', 0.2, 0.5, step=0.1)))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer =keras.optimizers.Adam(trial.suggest_float('lr',  1e-5, 1e-2, log=True)), loss='binary_crossentropy', metrics=['auc'])

    return model


In [ ]:
EPOCHS = 20
BATCH_SIZE = 10

def objective(trial):

    model = create_model(trial)

    model.fit(X_train, y_train, validation_data = (X_val, y_val), batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=False)

    _, accuracy = model.evaluate(X_val, y_val, verbose=0)

    return accuracy

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print(f"Best hyperparameters: {study.best_params}")
print(f"Best scores found: {study.best_value}")

params = []

for key, value in trial.params.items():
    params.append(value)
    print("    {}: {}".format(key, value))

[I 2024-10-19 10:57:55,232] A new study created in memory with name: no-name-cf9ddee7-0758-47b0-80a0-3b2517bafde0
[I 2024-10-19 11:00:09,231] Trial 0 finished with value: 0.9854685068130493 and parameters: {'num_hidden_layers': 1, 'units_1': 64, 'lr_l2_1': 1.0877873896027841e-05, 'dropout_1': 0.4, 'lr': 7.127820730729079e-05}. Best is trial 0 with value: 0.9854685068130493.
[I 2024-10-19 11:02:21,227] Trial 1 finished with value: 0.9814470410346985 and parameters: {'num_hidden_layers': 1, 'units_1': 32, 'lr_l2_1': 0.000174447948910878, 'dropout_1': 0.2, 'lr': 0.00798783466067626}. Best is trial 0 with value: 0.9854685068130493.
[I 2024-10-19 11:05:16,981] Trial 2 finished with value: 0.9814420938491821 and parameters: {'num_hidden_layers': 3, 'units_1': 64, 'lr_l2_1': 0.00042081708994635357, 'dropout_1': 0.2, 'units_2': 64, 'lr_l2_2': 0.00010034806695704537, 'dropout_2': 0.5, 'units_3': 32, 'lr_l2_3': 0.001389143112875942, 'dropout_3': 0.5, 'lr': 0.004035121240455675}. Best is trial 0 

Number of finished trials: 20
Best trial:
  Value: 0.9859735369682312
Best hyperparameters: {'num_hidden_layers': 2, 'units_1': 128, 'lr_l2_1': 0.00969753221848345, 'dropout_1': 0.2, 'units_2': 64, 'lr_l2_2': 1.008839517511501e-05, 'dropout_2': 0.30000000000000004, 'lr': 2.996387003849795e-05}
Best scores found: 0.9859735369682312
    num_hidden_layers: 2
    units_1: 128
    lr_l2_1: 0.00969753221848345
    dropout_1: 0.2
    units_2: 64
    lr_l2_2: 1.008839517511501e-05
    dropout_2: 0.30000000000000004
    lr: 2.996387003849795e-05


Los parámetros anteriores fueron los mejores hiperparámetros encontrados por Optuna, siendo una red de 2 capas ocultas, cada una con 128 y 64 neuronas.

In [ ]:
best_model = Sequential()

input_shape = (X_train.shape[1],)
best_model.add(Input(shape=input_shape))

best_model.add(Dense(units=128, activation='relu', kernel_initializer='random_normal',  kernel_regularizer=regularizers.L2(0.00969753221848345), bias_regularizer=regularizers.L2(0.00969753221848345),activity_regularizer=regularizers.L2(0.00969753221848345)))
best_model.add(Dropout(0.2))
best_model.add(Dense(units=64, activation='relu', kernel_initializer='random_normal',  kernel_regularizer=regularizers.L2(1.008839517511501e-05), bias_regularizer=regularizers.L2(1.008839517511501e-05),activity_regularizer=regularizers.L2(1.008839517511501e-05)))
best_model.add(Dropout(0.3))
best_model.add(Dense(1, activation='sigmoid'))

best_model.compile(optimizer =keras.optimizers.Adam(learning_rate=2.996387003849795e-05), loss='binary_crossentropy', metrics=['auc'])



In [ ]:
best_model.fit(X_train, y_train, batch_size = BATCH_SIZE,  validation_data = (X_val, y_val),
               epochs=EPOCHS,
               verbose=1)

y_pred = best_model.predict(X_test)

precision_gradient = roc_auc_score(y_test, y_pred)
print("AUC:", precision_gradient)

Epoch 1/20
4503/4503 ━━━━━━━━━━━━━━━━━━━━ 84s 14ms/step - auc: 0.8743 - loss: 0.7678 - val_auc: 0.9800 - val_loss: 0.3576
Epoch 2/20
4503/4503 ━━━━━━━━━━━━━━━━━━━━ 35s 4ms/step - auc: 0.9773 - loss: 0.3304 - val_auc: 0.9837 - val_loss: 0.2466
Epoch 3/20
4503/4503 ━━━━━━━━━━━━━━━━━━━━ 26s 6ms/step - auc: 0.9811 - loss: 0.2485 - val_auc: 0.9846 - val_loss: 0.2145
Epoch 4/20
4503/4503 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - auc: 0.9820 - loss: 0.2220 - val_auc: 0.9851 - val_loss: 0.1992
Epoch 5/20
4503/4503 ━━━━━━━━━━━━━━━━━━━━ 24s 3ms/step - auc: 0.9822 - loss: 0.2096 - val_auc: 0.9853 - val_loss: 0.1901
Epoch 6/20
4503/4503 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - auc: 0.9829 - loss: 0.1993 - val_auc: 0.9853 - val_loss: 0.1843
Epoch 7/20
4503/4503 ━━━━━━━━━━━━━━━━━━━━ 31s 6ms/step - auc: 0.9832 - loss: 0.1937 - val_auc: 0.9855 - val_loss: 0.1796
Epoch 8/20
4503/4503 ━━━━━━━━━━━━━━━━━━━━ 34s 4ms/step - auc: 0.9830 - loss: 0.1903 - val_auc: 0.9855 - val_loss: 0.1765
Epoch 9/20
4503/4503 ━━━━━━━━━━

Como se puede comprobar, esta red neuronal de Keras obtiene el resultado más alto de todos, llegando a un AUC considerablemente alto. Por lo tanto, se ha decidido escoger este modelo para utilizarlo como modelo final en la competición.

## **Ensembles**

Tras evaluar los modelos de aprendizaje automático y ajustar los mejores hiperparámetros para cada uno de ellos, procedemos a probar diversas técnicas de ensembles.

Los modelos **ensembles** combinan múltiples clasificadores inidviduales para mejorar tanto la precisión como la estabilidad de las predicciones. Esta combinación ayuda a mitigar los problemas asociados a los modelos individuales, como el **sesgo** y la **varianza**, generando predicciones más robustas y menos susceptibles a los errores cometidos por cada modelo.

El **objetivo** es realizar pruebas con distintos modelos de ensembles, tanto  **homogéneos** como **heterogéneos**, para intentar mejorar los resultados obtenidos previamente.


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import VotingClassifier

### **Ensembles homogéneos**

Los ensembles **homogéneos** son aquellos que se construyen utilizando un único tipo de modelo base para realizar las predicciones. Estos ensembles combinan las salidas de múltiples instancias del mismo clasificador, que pueden estar modificadas en su implementación o entrenadas con diferentes subconjuntos de datos.

Ejemplos comunes de ensembles homogéneos incluyen técnicas como:

*   **Bagging** (Bootstrap Aggregating): Genera múltiples versiones del modelo base, cada una entrenada con subconjuntos seleccionados aleatoriamente de los datos de entrenamiento y con reemplazo. Debido a esta variabilidad en los datos de entrenamiento, cada modelo es único y produce resultados diferentes. Las predicciones se combinan generalmente mediante votación o promediado, lo que contribuye a mitigar la varianza de las predicciones finales.

*   **Boosting**: En contraste con el Bagging, esta técnica entrena los modelos de manera secuencial, donde cada nuevo modelo se centra en corregir los errores del anterior, lo que permite una mejora continua en el rendimiento.



#### **Gradient Boosting**

El **Gradient Boosting** utiliza el árbol de decisión como modelo base. Esta técnica se basa en el entrenamiento secuencial de múltiples modelos débiles, donde cada nuevo modelo se centra en corregir los errores cometidos por el anterior. De esta manera, se busca mejorar continuamente el rendimiento del ensemble al abordar las deficiencias de los modelos previos.

El proceso de **Gradient Boosting** comienza con un modelo inicial que realiza predicciones basadas en los datos de entrenamiento. A continuación, se calculan los errores de este modelo, conocidos como **residuos**. Basándose en estos residuos, se generan otros **modelos débiles**, que suelen ser árboles de decisión de poca profundidad, entrenados específicamente para predecir los residuos. A partir de las predicciones de estos modelos débiles, se construye un nuevo modelo que integra estas predicciones, buscando así minimizar los residuos y mejorar la precisión general del ensemble.

*   **min_samples_split**: número mínimo de muestras necesarias para dividir un nodo en el árbol de decisión
*   **min_samples_leaf**: número mínimo de muestras que debe estar presente en una hoja  del árbol.
*   **loss**: función de pérdida que el modelo debe minimizar
*   **n_estimators**: número de árboles que se deben entrenar en el ensemble



In [ ]:
gradient = GradientBoostingClassifier(random_state = 42)
param_grid_gradient = {"min_samples_split": [1,2,3,4,5],
            "min_samples_leaf": [1,2,3,4,5],
            "loss": ['log_loss','exponential'],
            "n_estimators": [50,100,150,500,1000],
            }

gradient_random = RandomizedSearchCV(estimator = gradient, param_distributions = param_grid_gradient, n_iter = 20, scoring = 'roc_auc', cv = 5, verbose=False)

# Entrenar el modelo con los datos de entrenamiento
gradient_random.fit(X_train, y_train.values.ravel())

# Hacer predicciones con los datos de prueba
y_pred_gradient = gradient_random.predict(X_test)

# Evaluar la precisión del modelo
precision_gradient = roc_auc_score(y_test, y_pred_gradient)
print("Mejor combinación de hiperparámetros:", gradient_random.best_params_)
print("AUC:", precision_gradient)

/home/pvltarife/miniconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
35 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
35 fits failed with the following error:
Traceback (most recent call last):
  File "/home/pvltarife/miniconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/pvltarife/miniconda3/lib/python3.11/site-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/home/pvltarife/miniconda3/lib/python3.11/site-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints

Mejor combinación de hiperparámetros: {'n_estimators': 1000, 'min_samples_split': 5, 'min_samples_leaf': 1, 'loss': 'exponential'}
AUC: 0.9132367807978374


#### **AdaBoost**

El **AdaBoost** (Adaptive Boosting) utiliza el árbol de decisión como modelo base. Esta técnica se centra en el entrenamiento secuencial de múltiples modelos, los cuales se entrenan sobre diferentes versiones del conjunto de datos de entrenamiento.

El proceso de **AdaBoost** inicia con un modelo base que realiza predicciones sobre el conjunto de entrenamiento. A continuación, se calculan los errores de este modelo y se ajustan los pesos de las muestras en función de esos errores. Se aumenta el peso de las muestras que han sido clasificadas erróneamente, lo que permite que los modelos siguientes se centren en corregir estas instancias difíciles. Este enfoque iterativo ayuda a mejorar la precisión del ensemble al abordar las deficiencias de los modelos previos.

*   **n_estimators**: número de árboles que se deben entrenar en el ensemble
*   **learning_rate**: tasa de aprendizaje



In [ ]:
ada = AdaBoostClassifier(random_state=42)

# Definir la grilla de parámetros
param_grid_ada = {
    "n_estimators": [50, 100, 150, 500, 1000],
    "learning_rate": [0.01, 0.1, 0.5, 1]
}

# RandomizedSearchCV para AdaBoost
ada_random = RandomizedSearchCV(estimator=ada, param_distributions=param_grid_ada, n_iter=20, scoring='roc_auc', cv=5, verbose=False)

# Entrenar el modelo
ada_random.fit(X_train, y_train.values.ravel())

# Hacer predicciones
y_pred_ada = ada_random.predict(X_test)

# Evaluar precisión
precision_ada = roc_auc_score(y_test, y_pred_ada)
print("Mejor combinación de hiperparámetros:", ada_random.best_params_)
print("AUC AdaBoost:", precision_ada)

/home/pvltarife/miniconda3/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


/home/pvltarife/miniconda3/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/pvltarife/miniconda3/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/pvltarife/miniconda3/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/pvltarife/miniconda3/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to ci

Mejor combinación de hiperparámetros: {'n_estimators': 1000, 'learning_rate': 1}
AUC AdaBoost: 0.9154635707944117


#### **Bagging**

El **Bagging** (Bootstrap Aggregating) es una técnica de ensemble que se basa en la creación de múltiples modelos a partir de un modelo base, en este caso un árbol de decisión. Este método se centra en reducir la varianza al combinar las predicciones de varios modelos independientes, cada uno entrenado en subconjuntos aleatorios del conjunto de datos original.

El proceso de **Bagging** comienza seleccionando aleatoriamente múltiples subconjuntos del conjunto de datos de entrenamiento, utilizando un **muestreo con reemplazo**. Cada uno de estos subconjuntos se utiliza para entrenar un modelo independiente. Como resultado, cada modelo aprende de diferentes variaciones de los datos, lo que permite capturar distintos patrones.

Una vez que todos los modelos han sido entrenados, sus predicciones se combinan, generalmente mediante votación. Este enfoque ayuda a mitigar el sobreajuste, proporcionando una predicción más robusta y estable.

*   **n_estimators**: número de árboles que se deben entrenar en el ensemble
*   **max_samples**: proporción del conjunto de datos de entrenamiento que se utilizará para entrenar cada modelo base
*   **max_features**: proporción de características que se utilizarán para entrenar cada modelo base
*   **boostrap**: muestreo con reemplazo


In [ ]:
bagging = BaggingClassifier(random_state=42)

# Definir la grilla de parámetros
param_grid_bagging = {
    "n_estimators": [10, 50, 100, 200],
    "max_samples": [0.5, 0.7, 1.0],
    "max_features": [0.5, 0.7, 1.0],
    "bootstrap": [True, False]
}

# RandomizedSearchCV para Bagging
bagging_random = RandomizedSearchCV(estimator=bagging, param_distributions=param_grid_bagging, n_iter=20, scoring='roc_auc', cv=5, verbose=False)

# Entrenar el modelo
bagging_random.fit(X_train, y_train.values.ravel())

# Hacer predicciones
y_pred_bagging = bagging_random.predict(X_test)

# Evaluar precisión
precision_bagging = roc_auc_score(y_test, y_pred_bagging)
print("Mejor combinación de hiperparámetros:", bagging_random.best_params_)
print("AUC Bagging:", precision_bagging)


Mejor combinación de hiperparámetros: {'n_estimators': 200, 'max_samples': 0.7, 'max_features': 0.5, 'bootstrap': False}
AUC Bagging: 0.8934655063008513


 **Baggging + MLP**: Se realizó una prueba adicional utilizando como modelo base el mejor clasificador individual obtenido, que en este caso es el MLP. Para esta prueba, se repitió la Randomized Search con los mismos parámetros. Sin embargo, los resultados obtenidos no lograron superar el rendimiento del modelo individual.

In [ ]:
mlp = MLPClassifier(solver = 'adam', activation = 'relu', hidden_layer_sizes= (20,), alpha = 0.05, learning_rate='adaptive')
bagging = BaggingClassifier(estimator = mlp, random_state=42)
# Definir la grilla de parámetros
param_grid_bagging = {
    "n_estimators": [10, 50, 100, 200],
    "max_samples": [0.5, 0.7, 1.0],
    "max_features": [0.5, 0.7, 1.0],
    "bootstrap": [True, False]
}

# RandomizedSearchCV para Bagging
bagging_random = RandomizedSearchCV(estimator=bagging, param_distributions=param_grid_bagging, n_iter=20, scoring='roc_auc', cv=5, verbose=False)

# Entrenar el modelo
bagging_random.fit(X_train, y_train.values.ravel())

# Hacer predicciones
y_pred_bagging = bagging_random.predict(X_test)

# Evaluar precisión
precision_bagging = roc_auc_score(y_test, y_pred_bagging)
print("Mejor combinación de hiperparámetros:", bagging_random.best_params_)
print("AUC Bagging:", precision_bagging)

/home/pvltarife/miniconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/pvltarife/miniconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/pvltarife/miniconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/pvltarife/miniconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/pvltarife/mini

Mejor combinación de hiperparámetros: {'n_estimators': 50, 'max_samples': 0.7, 'max_features': 1.0, 'bootstrap': False}
AUC Bagging: 0.924753718335738


#### **Random Forest**

El **Random Forest** es una técnica de ensemble que combina múltiples árboles de decisión entrenados de manera indepenediente. Esta metodología se basa en el principio de Bagging.

El proceso de **Random Forest** comienza generando varios árboles de decisión a partir de diferentes muestras del conjunto de entrenamiento. Estas muestras se obtienen mediante un muestreo aleatorio con reemplazo. Además, durante la creación de cada árbol, se selecciona aleatoriamente un subconjunto de características para determinar el mejor split en cada nodo, lo que añade aún más diversidad entre los árboles.

Una vez que todos los árboles han sido entrenados, sus predicciones se combinan, generalmente mediante votación. Este enfoque de agregación permite que **Random Forest** capte patrones complejos en los datos mientras mantiene una mayor estabilidad y generalización en comparación con un solo árbol de decisión.

*   **n_estimators**: número de árboles que se deben entrenar en el ensemble
*   **criterion**: calidad de división de un árbol
*   **max_samples**: proporción del conjunto de datos de entrenamiento que se utilizará para entrenar cada modelo base
*   **max_features**: proporción de características que se utilizarán para entrenar cada modelo base
*   **boostrap**: muestreo con reemplazo


In [ ]:
random_forest = RandomForestClassifier(random_state = 42)

# Lista de clasificadores a probar
param_grid_random_forest = {
    "n_estimators": [10, 50, 100, 200, 300, 500],
    "criterion": ['gini', 'entropy', 'log_loss'],
    "max_samples": [0.5, 0.7, 1.0],
    "max_features": [0.5, 0.7, 1.0],
    "bootstrap": [True, False]
}

# RandomizedSearchCV para Bagging
random_forest_random = RandomizedSearchCV(estimator=random_forest, param_distributions=param_grid_random_forest, n_iter=20, scoring='roc_auc', cv=5, verbose=False)

# Entrenar el modelo
random_forest_random.fit(X_train, y_train.values.ravel())

# Hacer predicciones
y_pred_random_forest = random_forest_random.predict(X_test)

# Evaluar precisión
precision_random_forest = roc_auc_score(y_test, y_pred_random_forest)
print("Mejor combinación de hiperparámetros:", random_forest_random.best_params_)
print("AUC Random Forest:", precision_random_forest)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
55 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
55 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py", line 433, in fit
    raise ValueError(
ValueError: `max_sample` cannot be set if `bootstrap=False`.

Mejor combinación de hiperparámetros: {'n_estimators': 200, 'max_samples': 0.7, 'max_features': 0.7, 'criterion': 'log_loss', 'bootstrap': True}
AUC Random Forest: 0.9102241798770256


### **Ensembles heterogéneos**

Los **ensembles heterogéneos** son aquellos que combinan diferentes tipos de modelos para realizar las predicciones. Esto les permite aprovechar las fortalezas de cada uno de ellos. Estos modelos pueden variar en su estructura, como árboles de decisión, máquinas de soporte vectorial, regresiones logísticas, entre otros, y pueden ser entrenados en el mismo conjunto de datos o en diferentes subconjuntos.

La combinación de diferentes modelos en un **ensemble heterogéneo** busca mejorar la precisión y la robustez de las predicciones, así como mitigar el sesgo presente en las predicciones de modelos individuales.

En en este trabajo, se han elegido implementar dos técnicas de ensembles heterogéneos: **Stacking** y **Voting**. Los modelos seleccionados para esta combinación son aquellos que han demostrado el mejor rendimiento en los experimentos previos, junto con los parámetros óptimos obtenidos a través de la Randomized Search.

*   *MLP* - 0.9258
*   *SGD* - 0.9218
*   *Regresión Logística* - 0.9211
*   *SVM* - 0.9202



####  **Stacking**



**Stacking** es una técnica de ensemble que combina múltiples modelos base para mejorar la precisión de las predicciones.

El proceso de **Stacking** comienza con el entrenamiento de múltiples modelos utilizando el mismo conjunto de datos de entrenamiento. Cada modelo proporciona una predicción, la cual se utiliza como entrada para un **meta modelo**. Este modelo meta, que generalmente es un clasificador más simple, se entrena para combinar las predicciones de los modelos base y mejorar así la precisión general del ensemble.

Al final del proceso, el modelo meta genera la predicción final integrando las salidas de los modelos base, lo que permite, en muchos casos, superar el rendimiento de cualquier modelo individual.

En este caso, se utiliza como meta modelo un Regresor Logístico y se realiza una Randomized Search para ajustar sus parámetros.

*   **C**: factor de regularización
*   **penalty**: penalización
*   **solver**: optimizador
*   **max_iter**: máximo de iteraciones que el algoritmo puede realizar durante el proceso de optimización
*   **stack_method**: método que se utilizará para combinar las predicciones de los modelos base en el modelo meta


In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# Definir los clasificadores base
estimators = [
    ('mlp', MLPClassifier(solver = 'adam', activation = 'relu', hidden_layer_sizes= (20,), alpha = 0.05, learning_rate='adaptive')),
    ('svc', LinearSVC(C= 0.1, dual= True, loss='squared_hinge', max_iter = 10000, penalty='l2', tol = 0.0001)),
    ('lr', LogisticRegression(C= 0.1, solver = 'liblinear', max_iter = 200, penalty='l1')),
    ('sgd', SGDClassifier(alpha=0.001, loss = 'hinge', max_iter = 5000, penalty = 'l2', tol = 0.01))
]
# Inicializar el modelo Stacking
stacking = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())

# Definir la grilla de parámetros
param_grid_stacking = {
    'final_estimator__C': [0.001, 0.01, 0.1, 1, 10, 100],
    'final_estimator__penalty': ['l1', 'l2', 'elasticnet'],
    'final_estimator__solver': ['liblinear', 'saga'],           # Solvers compatibles con l1 y elasticnet
    'final_estimator__max_iter': [100, 200, 300],
    'stack_method': ['auto', 'predict_proba', 'decision_function', 'predict']            # Más iteraciones si el dataset es grande
}

# RandomizedSearchCV para Stacking
stacking_random = RandomizedSearchCV(estimator=stacking, param_distributions=param_grid_stacking, n_iter=20, scoring='roc_auc', cv=5, verbose=False)

# Entrenar el modelo
stacking_random.fit(X_train, y_train.values.ravel())

# Hacer predicciones
y_pred_stacking = stacking_random.predict(X_test)

# Evaluar precisión
precision_stacking = roc_auc_score(y_test, y_pred_stacking)
print("Mejor combinación de hiperparámetros:", stacking_random.best_params_)
print("AUC Stacking:", precision_stacking)


/home/pvltarife/miniconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/pvltarife/miniconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/pvltarife/miniconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/pvltarife/miniconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/pvltarife/miniconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/pvltarife/miniconda3/lib/python3.11/sit

Mejor combinación de hiperparámetros: {'stack_method': 'auto', 'final_estimator__solver': 'saga', 'final_estimator__penalty': 'l2', 'final_estimator__max_iter': 100, 'final_estimator__C': 1}
AUC Stacking: 0.9236399324839696


/home/pvltarife/miniconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


#### **Voting**

**Voting** es una técnica de ensemble que combina las predicciones de múltiples modelos para producir una única predicción final.

Existen dos enfoques principales para el voting: hard voting y soft voting. En el *hard voting*, cada modelo da un voto por la clase que predice, y la clase que recibe la mayoría de los votos se elige como la predicción final. Por otro lado, el *soft voting* combina las probabilidades asignadas a cada clase por los modelos base, eligiendo la clase con la suma más alta de probabilidades como la predicción final.

*   **voting**: enfoque de combinación de predicciones

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

voting_estimators = [
    ('mlp', MLPClassifier(solver = 'adam', activation = 'relu', hidden_layer_sizes= (20,), alpha = 0.05, learning_rate='adaptive')),
    ('svc', LinearSVC(C= 0.1, dual= True, loss='squared_hinge', max_iter = 10000, penalty='l2', tol = 0.0001)),
    ('lr', LogisticRegression(C= 0.1, solver = 'liblinear', max_iter = 200, penalty='l1')),
    ('sgd', SGDClassifier(alpha=0.001, loss = 'hinge', max_iter = 5000, penalty = 'l2', tol = 0.01))
]

# Inicializar el modelo Voting
voting = VotingClassifier(estimators=voting_estimators, voting='soft')

# Definir la grilla de parámetros
param_grid_voting = {
    'voting': ['hard', 'soft']
}

# RandomizedSearchCV para Voting
voting_random = RandomizedSearchCV(estimator=voting, param_distributions=param_grid_voting, n_iter=20, scoring='roc_auc', cv=5, verbose=False)

# Entrenar el modelo
voting_random.fit(X_train, y_train.values.ravel())

# Hacer predicciones
y_pred_voting = voting_random.predict(X_test)

# Evaluar precisión
precision_voting = roc_auc_score(y_test, y_pred_voting)
print("Mejor combinación de hiperparámetros:", voting_random.best_params_)
print("AUC Voting:", precision_voting)


/home/pvltarife/miniconda3/lib/python3.11/site-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 2 is smaller than n_iter=20. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


/home/pvltarife/miniconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/pvltarife/miniconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/pvltarife/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/pvltarife/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 370, in _score
    response_method = _check_response_method(estimator, self._response_method)
                  

Mejor combinación de hiperparámetros: {'voting': 'hard'}
AUC Voting: 0.9205737602078714


### **Análisis de resultados**

Observando los resultados, notamos que ninguno de estos modelos de ensembles logró superar el rendimiento de los mejores clasificadores individuales. Incluso al combinar técnicas como stacking y voting, los resultados no alcanzaron los niveles esperados. Esto nos demuestra que, aunque los ensembles pueden ser efectivos en muchos casos, no siempre garantizan una mejora en todos los escenarios.



## **Conclusiones**

 En conclusión, el desempeño de los modelos de ensembles no fue comparable con el de la **red neuronal** implementada en Keras, la cual mostró un rendimiento significativamente superior a los demás modelos y técnicas utilizadas.



*   Esto refuerza la idea de que, en problemas complejos, modelos más avanzados como las redes neuronales tienen la **capacidad de capturar mejor las relaciones no lineales** en los datos y producir predicciones más precisas que otros enfoques, como los árboles de decisión o las regresiones lineales.

*   Al utilizar una red neuronal **sencilla y poco profunda**, con solo dos capas ocultas y técnicas como Dropout, logramos **evitar el sobreajuste**. Esto permitió mejorar la generalización y obtener mejores resultados en los datos de prueba.

Sin embargo, somos conscientes de que el uso de este tipo de modelos más avanzados también conlleva algunas desventajas:

*   Una desventaja importante es la **mayor complejidad** que implica entrenar redes neuronales, tanto en términos de **ajuste de hiperparámetros** como de **tiempo de entrenamiento**.


*   Además, la red presenta una **baja interpretabilidad** en comparación con otros modelos tradicionales de aprendizaje automático, lo que puede ser un reto si se necesita explicar detalladamente las predicciones.
